# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [4]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [23]:
# 1)

url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
resp = requests.get(url)

data_json = resp.json()
data_json['items']

[{'id_pleito': 7255,
  'tipo_interessado': 'Estado',
  'interessado': 'Rio de Janeiro',
  'cod_ibge': 33,
  'uf': 'RJ',
  'num_pvl': None,
  'status': 'Encaminhado à PGFN com manifestação técnica favorável',
  'num_processo': '17944.000532/2012-31',
  'data_protocolo': '2012-05-24T03:00:00Z',
  'tipo_operacao': 'Operação contratual interna (com garantia da União)',
  'finalidade': 'Infraestrutura',
  'tipo_credor': 'Instituição Financeira Nacional',
  'credor': 'Banco do Brasil S/A',
  'moeda': 'Real',
  'valor': 3645563000,
  'pvl_assoc_divida': 1,
  'pvl_contradado_credor': 0,
  'data_status': '29/05/2012'},
 {'id_pleito': 22594,
  'tipo_interessado': 'Estado',
  'interessado': 'Rio de Janeiro',
  'cod_ibge': 33,
  'uf': 'RJ',
  'num_pvl': '00000.000000/2031-89',
  'status': 'Arquivado por decurso de prazo',
  'num_processo': '17944.000657/2016-95',
  'data_protocolo': '2016-05-12T03:00:00Z',
  'tipo_operacao': 'Operação contratual interna (com garantia da União)',
  'finalidade': 'I

In [21]:
# 2)
df = pd.DataFrame(data_json['items'])
df['status'].value_counts()

Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [30]:
# 3)
df['data_status'].str.slice(6).value_counts()

2012    11
2009     9
2013     7
2011     6
2016     4
2002     4
2014     4
2010     3
2008     2
2006     2
Name: data_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [44]:
#1)

def get_pvl_data_frame(uf = 'SP', interessado='Estado'):
    url = f'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={interessado}'
    print(url)
    resp = requests.get(url,verify=False)
    data_json = resp.json()
    return pd.DataFrame(data_json['items'])

In [53]:
# 2)
df_mg = get_pvl_data_frame('MG')
df_mg[df_mg['status'] == 'Arquivado por decurso de prazo']['status'].value_counts()

http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado


C:\Users\Dell\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apidatalake.tesouro.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [68]:
# 3) Qual é o município da Bahia com mais solicitações deferidas?

df_ba = get_pvl_data_frame('BA', "Município")
df_ba[df_ba['status'] == 'Deferido'][['interessado', 'status']].value_counts()


http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Município


C:\Users\Dell\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apidatalake.tesouro.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


interessado             status  
Luís Eduardo Magalhães  Deferido    14
Vitória da Conquista    Deferido    11
Lauro de Freitas        Deferido    10
Camaçari                Deferido    10
Barreiras               Deferido     9
                                    ..
Licínio de Almeida      Deferido     1
Lençóis                 Deferido     1
Boa Vista do Tupim      Deferido     1
Jeremoabo               Deferido     1
Xique-Xique             Deferido     1
Length: 176, dtype: int64

In [74]:
# 4)

df_ba_to_save = get_pvl_data_frame('BA')
df.to_csv('pvl_bahia.csv', sep=';', decimal=',', encoding='utf-8-sig')

http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Estado


C:\Users\Dell\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apidatalake.tesouro.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
